## Imports

In [1]:
from qiskit import *
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram
from qiskit.providers.aer import QasmSimulator
import qiskit.quantum_info as qi
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_bloch_multivector, plot_histogram

import numpy as np
import math
import random

%matplotlib inline

from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
from matplotlib import cm

## Dataset generation

Output of the old generator
```python
[(2,
  array([45, 11, 40, 38]),
  {(0, 2): -15, (0, 3): -9, (1, 2): -9, (1, 3): -14}),
 (2,
  array([10, 37,  9, 46]),
  {(0, 2): -20, (0, 3): -15, (1, 2): -2, (1, 3): -4}),
 (2,
  array([12, 48, 35, 38]),
  {(0, 2): -3, (0, 3): -7, (1, 2): -19, (1, 3): -8}),
 (2,
  array([ 4, 42, 28, 33]),
  {(0, 2): -11, (0, 3): -10, (1, 2): -1, (1, 3): -14}),
 (2,
  array([43, 23, 23, 18]),
  {(0, 2): -3, (0, 3): -2, (1, 2): -16, (1, 3): 0})]
  ```

In [2]:
def create_savings(n_queries, n_plans_per_query):
    savings = {}
    for i in range(n_queries-1):
        for j in range(n_plans_per_query[i]):
            s = j + np.sum(n_plans_per_query[0:i], dtype=int)
            for a in range(i+1, n_queries):
                for b in range(n_plans_per_query[a]):
                    t = b + np.sum(n_plans_per_query[:a], dtype=int)
                    savings[s, t] = random.randint(-20, 0)

    return savings

In [27]:
def create_problems(n_problems, n_queries, n_plans_per_query, cost_min = 1, cost_max = 50, savings_min = -20, savings_max = 0):
    problems = []
    for i in range(n_problems):
        problems.append((n_plans_per_query, np.random.randint(cost_min, cost_max, np.sum(n_plans_per_query)), 
            create_savings(n_queries, n_plans_per_query)))
    return problems

Problems are generated, but only work, for now, in double combinations -> no three way savings!

In [108]:
problems = create_problems(1, 4, [2,2,2,2])
problems

[([2, 2, 2, 2],
  array([41, 20, 25, 19, 30, 32, 15,  6]),
  {(0, 2): -8,
   (0, 3): -9,
   (0, 4): -7,
   (0, 5): -19,
   (0, 6): -5,
   (0, 7): 0,
   (1, 2): -7,
   (1, 3): -7,
   (1, 4): -18,
   (1, 5): -7,
   (1, 6): -19,
   (1, 7): -3,
   (2, 4): -7,
   (2, 5): -6,
   (2, 6): -4,
   (2, 7): -11,
   (3, 4): -10,
   (3, 5): -16,
   (3, 6): -3,
   (3, 7): -15,
   (4, 6): -12,
   (4, 7): 0,
   (5, 6): -3,
   (5, 7): -8})]

We now want to classically solve the problem so we can get a ranking.

In [109]:
comb_savings_per_problem = []
for problem in problems:
    current_combinations = problem[2]
    while len(current_combinations) > np.prod(problem[0]):
        total_savings = {}
        for a in current_combinations:
            saves = current_combinations[a]
            for b in [z for z in problem[2] if z[0] == a[-1]]:
                c = list(a)
                c.append(b[-1])
                c = tuple(c)
                total_savings[c] = saves + sum([problem[2][x, b[-1]] for x in a ])
        current_combinations = total_savings
    comb_savings_per_problem.append(current_combinations)
print(comb_savings_per_problem)

[{(0, 2, 4, 6): -43, (0, 2, 4, 7): -33, (0, 2, 5, 6): -45, (0, 2, 5, 7): -52, (0, 3, 4, 6): -46, (0, 3, 4, 7): -41, (0, 3, 5, 6): -55, (0, 3, 5, 7): -67, (1, 2, 4, 6): -67, (1, 2, 4, 7): -46, (1, 2, 5, 6): -46, (1, 2, 5, 7): -42, (1, 3, 4, 6): -69, (1, 3, 4, 7): -53, (1, 3, 5, 6): -55, (1, 3, 5, 7): -56}]


In [59]:
#def solve_classical(problems):
combi_costs = []
for problem in problems:
    savings = problem[2]
    while len(savings) > np.prod(problem[0]):
        new_savings = {}
        for a in savings:
            for b in [z for z in savings if z[0] == a[-1]]:
                c = list(a)
                c.append(b[-1])
                new_savings[tuple(c)] = savings[a] + savings[b] + sum(x for x in a[1:])
        if new_savings == savings:
            savings = new_savings
            break
        else:
            savings = new_savings
            print(savings)
    combi_costs.append(savings)

#print(combi_costs[0])
#   return solutions_complete

{(0, 2, 4): -22, (0, 2, 5): -21, (0, 2, 6): -11, (0, 2, 7): -5, (0, 3, 4): -5, (0, 3, 5): -18, (0, 3, 6): -8, (0, 3, 7): -10, (0, 4, 6): -23, (0, 4, 7): -22, (0, 5, 6): -22, (0, 5, 7): -4, (1, 2, 4): -35, (1, 2, 5): -34, (1, 2, 6): -24, (1, 2, 7): -18, (1, 3, 4): -10, (1, 3, 5): -23, (1, 3, 6): -13, (1, 3, 7): -15, (1, 4, 6): -7, (1, 4, 7): -6, (1, 5, 6): -28, (1, 5, 7): -10, (2, 4, 6): -23, (2, 4, 7): -22, (2, 5, 6): -30, (2, 5, 7): -12, (3, 4, 6): -7, (3, 4, 7): -6, (3, 5, 6): -28, (3, 5, 7): -10}
{}


In [7]:
for i, costs in enumerate(combi_costs):
    for a in costs:
        for b in a:
            costs[a] += problems[0][1][b]
    combi_costs[i] = costs
costs


{(0, 2, 5): 34,
 (0, 2, 6): 78,
 (0, 3, 5): -21,
 (0, 3, 6): 23,
 (0, 4, 5): 16,
 (0, 4, 6): 52,
 (1, 2, 5): 33,
 (1, 2, 6): 77,
 (1, 3, 5): -11,
 (1, 3, 6): 33,
 (1, 4, 5): 31,
 (1, 4, 6): 67}

In [8]:
bit_strings = []
for a in costs:
    b = list('0'*sum(problems[0][0]))
    for i in a:
        b[i] = '1'
    bit_strings.append(''.join(b))
bit_strings

['1010010',
 '1010001',
 '1001010',
 '1001001',
 '1000110',
 '1000101',
 '0110010',
 '0110001',
 '0101010',
 '0101001',
 '0100110',
 '0100101']

In [9]:
combi_costs[0]

{(0, 2, 5): 34,
 (0, 2, 6): 78,
 (0, 3, 5): -21,
 (0, 3, 6): 23,
 (0, 4, 5): 16,
 (0, 4, 6): 52,
 (1, 2, 5): 33,
 (1, 2, 6): 77,
 (1, 3, 5): -11,
 (1, 3, 6): 33,
 (1, 4, 5): 31,
 (1, 4, 6): 67}

In [10]:
#sort costs so theyre... sorted by cheapest first

for i, cost in enumerate(combi_costs):
    combi_costs[i] = {k: cost[k] for k in sorted(cost, key=cost.get)}

combi_costs


[{(0, 3, 5): -21,
  (1, 3, 5): -11,
  (0, 4, 5): 16,
  (0, 3, 6): 23,
  (1, 4, 5): 31,
  (1, 2, 5): 33,
  (1, 3, 6): 33,
  (0, 2, 5): 34,
  (0, 4, 6): 52,
  (1, 4, 6): 67,
  (1, 2, 6): 77,
  (0, 2, 6): 78}]

In [11]:
solution_keys_ranking = []
for cost in combi_costs:
    bit_strings = []
    for a in cost:
        b = list('0'*sum(problems[0][0]))
        for i in a:
            b[i] = '1'
        bit_strings.append(''.join(b))
    solution_keys_ranking.append(bit_strings)
solution_keys_ranking
    

[['1001010',
  '0101010',
  '1000110',
  '1001001',
  '0100110',
  '0110010',
  '0101001',
  '1010010',
  '1000101',
  '0100101',
  '0110001',
  '1010001']]

We now generate the combinational bitstrings that are possible

In [12]:
n_qubits = np.sum(problems[0][0])
binary_string = []
for i, v in enumerate(problems[0][0]):
    if i == 0:
        for j in range(v):
            binary_string.append('0'*j + '1' + '0'*(v-j-1))
    else:
        copy = []
        for x in binary_string:
            for j in range(v):
                copy.append(x + '0'*j + '1' + '0'*(v-j-1))
        binary_string = copy
print(binary_string)


['1010010', '1010001', '1001010', '1001001', '1000110', '1000101', '0110010', '0110001', '0101010', '0101001', '0100110', '0100101']


Now we generate the circuits...

In [13]:
from qiskit import *
from qiskit import Aer
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram

import numpy as np
import math

In [14]:
problems = create_problems(1, 3, [2,3,2])
problems

[([2, 3, 2],
  array([21,  1, 36, 16, 33, 42,  7]),
  {(0, 2): -7,
   (0, 3): -13,
   (0, 4): -14,
   (0, 5): -6,
   (0, 6): 0,
   (1, 2): -18,
   (1, 3): -12,
   (1, 4): -17,
   (1, 5): -14,
   (1, 6): -4,
   (2, 5): -5,
   (2, 6): -6,
   (3, 5): -20,
   (3, 6): -12,
   (4, 5): -10,
   (4, 6): 0})]

In [15]:
circuit = QuantumCircuit(np.sum(problems[0][0]))
circuit.h(range(circuit.width()))
for i, v in enumerate(problems[0][1]):
    circuit.ry(v, i)
circuit.barrier()

prev_i = 0
for i, v in problems[0][2]:
    if prev_i != i:
        circuit.barrier()
    circuit.crz(problems[0][2][i, v], i, v)
    prev_i = i
circuit.barrier()

circuit.draw()

┌───┐┌────────┐ ░                                                     ░ »
q_0: ┤ H ├┤ Ry(21) ├─░─────■──────────■──────────■─────────■─────────■─────░─»
     ├───┤├───────┬┘ ░     │          │          │         │         │     ░ »
q_1: ┤ H ├┤ Ry(1) ├──░─────┼──────────┼──────────┼─────────┼─────────┼─────░─»
     ├───┤├───────┴┐ ░ ┌───┴────┐     │          │         │         │     ░ »
q_2: ┤ H ├┤ Ry(36) ├─░─┤ Rz(-7) ├─────┼──────────┼─────────┼─────────┼─────░─»
     ├───┤├────────┤ ░ └────────┘┌────┴────┐     │         │         │     ░ »
q_3: ┤ H ├┤ Ry(16) ├─░───────────┤ Rz(-13) ├─────┼─────────┼─────────┼─────░─»
     ├───┤├────────┤ ░           └─────────┘┌────┴────┐    │         │     ░ »
q_4: ┤ H ├┤ Ry(33) ├─░──────────────────────┤ Rz(-14) ├────┼─────────┼─────░─»
     ├───┤├────────┤ ░                      └─────────┘┌───┴────┐    │     ░ »
q_5: ┤ H ├┤ Ry(42) ├─░─────────────────────────────────┤ Rz(-6) ├────┼─────░─»
     ├───┤├───────┬┘ ░                                 └────────┘┌───┴───┐ ░ »
q_6: ┤ H ├┤ Ry(7) ├──░───────────────────────────────────────────┤ Rz(0) ├─░─»
     └───┘└───────┘  ░                                           └───────┘ ░ »
«                                                            ░           »
«q_0: ───────────────────────────────────────────────────────░───────────»
«                                                            ░           »
«q_1: ─────■──────────■──────────■──────────■─────────■──────░───────────»
«     ┌────┴────┐     │          │          │         │      ░           »
«q_2: ┤ Rz(-18) ├─────┼──────────┼──────────┼─────────┼──────░─────■─────»
«     └─────────┘┌────┴────┐     │          │         │      ░     │     »
«q_3: ───────────┤ Rz(-12) ├─────┼──────────┼─────────┼──────░─────┼─────»
«                └─────────┘┌────┴────┐     │         │      ░     │     »
«q_4: ──────────────────────┤ Rz(-17) ├─────┼─────────┼──────░─────┼─────»
«                           └─────────┘┌────┴────┐    │      ░ ┌───┴────┐»
«q_5: ─────────────────────────────────┤ Rz(-14) ├────┼──────░─┤ Rz(-5) ├»
«                                      └─────────┘┌───┴────┐ ░ └────────┘»
«q_6: ────────────────────────────────────────────┤ Rz(-4) ├─░───────────»
«                                                 └────────┘ ░           »
«                ░                        ░                      ░ 
«q_0: ───────────░────────────────────────░──────────────────────░─
«                ░                        ░                      ░ 
«q_1: ───────────░────────────────────────░──────────────────────░─
«                ░                        ░                      ░ 
«q_2: ────■──────░────────────────────────░──────────────────────░─
«         │      ░                        ░                      ░ 
«q_3: ────┼──────░──────■──────────■──────░──────────────────────░─
«         │      ░      │          │      ░                      ░ 
«q_4: ────┼──────░──────┼──────────┼──────░──────■─────────■─────░─
«         │      ░ ┌────┴────┐     │      ░ ┌────┴────┐    │     ░ 
«q_5: ────┼──────░─┤ Rz(-20) ├─────┼──────░─┤ Rz(-10) ├────┼─────░─
«     ┌───┴────┐ ░ └─────────┘┌────┴────┐ ░ └─────────┘┌───┴───┐ ░ 
«q_6: ┤ Rz(-6) ├─░────────────┤ Rz(-12) ├─░────────────┤ Rz(0) ├─░─
«     └────────┘ ░            └─────────┘ ░            └───────┘ ░

In [16]:
np.random.randint(1,4, size=5)

array([2, 1, 3, 3, 3])